## Init and import

In [1]:
# @title Imports

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='llama2:70b',
    # streaming=True
)

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [5]:
#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [6]:
# @title Generic memories are memories that all players and GM share.

shared_memories = []

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
# shared_context = model.sample_text(
#     'Summarize the following passage in a concise and insightful fashion:\n'
#     + '\n'.join(shared_memories)
#     + '\n'
#     + 'Summary:'
# )
# print(shared_context)


## Configure and build the players

---



In [7]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [8]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 1

scenario_premise = []
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice wants to be a successful carpenter and a good person.',
        context='Alice has been a carpenter for most of her career. She has also worked as a music store clerk and a waitress. She is a good person.',
        traits='',
    ),
]

In [9]:
def build_agent(
    agent_config,
    player_names: list[str],
    measurements: measurements_lib.Measurements | None = None,
):
  mem = formative_memory_factory.make_memories(agent_config)

  self_perception = components.self_perception.SelfPerception(
      name='self perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  persona = components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
      ],
  )
  current_time_component = components.report_function.ReportFunction(
      name='current_time', function=clock.current_time_interval_str
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )

  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      components=[persona],
      component_name='summary of observations',
  )

  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )

  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
          persona,
          current_time_component,
          summary_obs,
          current_obs,
          goal_metric,
      ],
      update_interval=time_step,
  )

  return agent

In [10]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

In [12]:
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)

quiz_metric = qm.QuizMetric(model, "Alice", clock, "./custom_components/data_structures_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(f"Pretend that you have been a carpenter for your whole life. You have never learned to code. You don't know anything about data structures. How would you answer the following question?")

Question: Which of the following data structures uses a FIFO approach?, Agent answered correctly: Queue
Question: What is the time complexity of accessing an element in an array?, Agent answered correctly: O(1)
Question: In a binary search tree, the left child of a node contains a value which is:, Agent answered correctly: Less than the parent node
Question: Which of the following is not a linear data structure?, Agent answered incorrectly: Linked List
Question: What data structure is used for implementing recursion?, Agent answered incorrectly: Array
Question: Which of the following sorting algorithms is not a comparison sort?, Agent answered incorrectly: Insertion Sort
Question: What is the worst-case time complexity of a binary search algorithm?, Agent answered incorrectly: O(n^2)
Question: Which of the following is an example of a dynamic data structure?, Agent answered correctly: Linked List
Question: What is the maximum number of children for a node in a binary tree?, Agent answe

In [13]:
quiz_metric = qm.QuizMetric(model, "Bob", clock, "./custom_components/data_structures_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(f"How would you answer the following question?")

Question: Which of the following data structures uses a FIFO approach?, Agent answered correctly: Queue
Question: What is the time complexity of accessing an element in an array?, Agent answered correctly: O(1)
Question: In a binary search tree, the left child of a node contains a value which is:, Agent answered correctly: Less than the parent node
Question: Which of the following is not a linear data structure?, Agent answered correctly: Graph
Question: What data structure is used for implementing recursion?, Agent answered incorrectly: Array
Question: Which of the following sorting algorithms is not a comparison sort?, Agent answered incorrectly: Merge Sort
Question: What is the worst-case time complexity of a binary search algorithm?, Agent answered incorrectly: O(n^2)
Question: Which of the following is an example of a dynamic data structure?, Agent answered incorrectly: Array
Question: What is the maximum number of children for a node in a binary tree?, Agent answered incorrectly: